# Examples

with some unscientific performance timings

In [1]:
%load_ext watermark

In [2]:
%watermark -m -v -p duckdb

Python implementation: CPython
Python version       : 3.10.2
IPython version      : 8.0.1

duckdb: 0.3.3.dev1397

Compiler    : Clang 12.0.5 (clang-1205.0.22.9)
OS          : Darwin
Release     : 21.4.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [3]:
from pathlib import Path
import time

import duckdb
import pandas as pd
import seaborn as sns

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)

sns.set_style('whitegrid')

In [4]:
QUERY_DIR = Path.cwd() / 'queries'

In [5]:
def load_query(query_name: str, show: bool = False) -> str:
    fname = QUERY_DIR / query_name
    fname = fname.with_suffix('.sql') if not query_name.endswith('sql') else fname
    
    with open(fname) as f:
        query = f.read()
    
    if show:
        print(query)

    return query

In [6]:
conn = duckdb.connect('../lahnman.duckdb')

## Longest tenure by team

Which players had the longest tenure for each franchise? For this query we are only going to consider currently active franchises.

In [7]:
longest_tenure_query = load_query('longest_tenure_by_team', show=True)

WITH player_tenures AS (
SELECT
    a.playerID
    ,p.nameFirst
    ,p.nameLast
    ,t.franchID
    ,t.name as team_name
    ,MIN(a.yearID) as first_year
    ,MAX(a.yearID) as last_year
    ,COUNT(DISTINCT a.yearID) as years
    ,DENSE_RANK() OVER (PARTITION BY t.franchID ORDER BY COUNT(DISTINCT a.yearID) DESC) as rank
FROM appearances a
JOIN people p
    ON a.playerID = p.playerID
JOIN teams t
    ON t.teamID = a.teamID
    AND t.yearID = a.yearID
JOIN teamsfranchises tf
    ON t.franchID = tf.franchID
    AND t.name = tf.franchName
WHERE
    tf.active = 'Y'
GROUP BY
    a.playerID
    ,p.nameFirst
    ,p.nameLast
    ,t.franchID
    ,t.name
)

SELECT
    playerID
    ,nameFirst
    ,nameLast
    ,franchID
    ,team_name
    ,first_year
    ,last_year
    ,years
FROM player_tenures
WHERE
    rank = 1
ORDER BY
    franchID
    ,years DESC



In [8]:
start = time.perf_counter()
longest_tenure = conn.execute(longest_tenure_query).df()
print(f'query time: {time.perf_counter() - start:.3}')

# make sure all teams are represented
current_distinct_teams = conn.execute("SELECT DISTINCT franchID FROM teamsfranchises WHERE active = 'Y'").fetchall()
assert len(current_distinct_teams) == len(longest_tenure['franchID'].unique())

longest_tenure

query time: 0.0749


,playerID,nameFirst,nameLast,franchID,team_name,first_year,last_year,years
0,weaveje02,Jered,Weaver,ANA,Los Angeles Angels of Anaheim,2006,2016,11
1,troutmi01,Mike,Trout,ANA,Los Angeles Angels of Anaheim,2011,2021,11
2,montemi01,Miguel,Montero,ARI,Arizona Diamondbacks,2006,2014,9
3,smoltjo01,John,Smoltz,ATL,Atlanta Braves,1988,2008,20
4,robinbr01,Brooks,Robinson,BAL,Baltimore Orioles,1955,1977,23
5,yastrca01,Carl,Yastrzemski,BOS,Boston Red Sox,1961,1983,23
6,cavarph01,Phil,Cavarretta,CHC,Chicago Cubs,1934,1953,20
7,lyonste01,Ted,Lyons,CHW,Chicago White Sox,1923,1946,21
8,rosepe01,Pete,Rose,CIN,Cincinnati Reds,1963,1986,19
9,conceda01,Dave,Concepcion,CIN,Cincinnati Reds,1970,1988,19
